In [1]:
import os
import sys
import pyxrt
import contextlib
from neuroring import NeuroRingHost, NeuroRingKernel
import nest
import network
import numpy as np
from network_params import net_dict
from sim_params import sim_dict
from stimulus_params import stim_dict
from utils_binding import Options  # Ensure this module provides .index and .bitstreamFile

net = network.Network(sim_dict, net_dict, stim_dict)
net.create()
net.connect()

print(net.pops)

XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 2691035
UID: 1031
[Thu Aug  7 09:28:15 2025 GMT]
HOST: cyberdeck
EXE: /usr/bin/orted
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
[XRT] ERROR: clGetDeviceInfo: invalid param_name
XRT build version: 2.18.179
Build hash: 3ade2e671e5ab463400813fc2846c57edf82bb10
Build date: 2024-11-05 13:57:47
Git branch: 2024.2
PID: 2690938
UID: 1031
[Thu Aug  7 09:28:15 2025 GMT]
HOST


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.8.0
 Built: Jan 27 2025 08:18:03

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



/home/miahafiz/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning:Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
/home/miahafiz/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning:Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
/home/miahafiz/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning:Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
/home/miahafiz/NeuroRing/host_py/network.py:210: UserWarning:DC input created to compensate missing Poisson input.



Data will be written to: /home/miahafiz/NeuroRing/host_py/data/
  Directory already existed. Old data will be overwritten.

Neuron numbers are scaled by a factor of 0.010.
Indegrees are scaled by a factor of 0.010.
  Weights and DC input are adjusted to compensate.


Aug 07 11:28:18 SimulationManager::set_status [Info]: 
    Temporal resolution changed from 0.1 to 0.1 ms.
RNG seed: 55
Total number of virtual processes: 1
Creating neuronal populations.
Creating recording devices.
  Creating spike recorders.
Creating DC generators for external stimulation.
Connecting neuronal populations recurrently.
Connecting DC generators.
[NodeCollection(metadata=None, model=iaf_psc_exp, size=207, first=1, last=207), NodeCollection(metadata=None, model=iaf_psc_exp, size=58, first=208, last=265), NodeCollection(metadata=None, model=iaf_psc_exp, size=219, first=266, last=484), NodeCollection(metadata=None, model=iaf_psc_exp, size=55, first=485, last=539), NodeCollection(metadata=None, model=iaf_psc_exp

In [2]:
host = NeuroRingHost(net, 4, 1, "/home/miahafiz/NeuroRing/krnl_neuroring_hw.xclbin")
host.initialize_devices()

Extracting synapse information...
Initialized device 0 with XCLBIN UUID: c2137429-da21-dc88-f242-71af04b87388
threshold: -50.0
membrane_potential: -65.0
dcstim_amp: 209.11304111722728
{'simulation_time': 1, 'threshold': 3259498496, 'membrane_potential': 3263299584, 'amount_of_cores': 4, 'neuron_start': 1, 'neuron_total': 192, 'dcstim_start': 0, 'dcstim_total': 1000, 'dcstim_amp': 1129389296, 'device': None, 'xclbin': None, 'uuid': None, 'kernel_name': None, 'kernel': None}
Initialized kernel NeuroRing:{NeuroRing_0} and AxonLoader:{AxonLoader_0} on device <pyxrt.device object at 0x7f9bf5b8c2f0>
threshold: -50.0
membrane_potential: -65.0
dcstim_amp: 209.11304111722728
{'simulation_time': 1, 'threshold': 3259498496, 'membrane_potential': 3263299584, 'amount_of_cores': 4, 'neuron_start': 193, 'neuron_total': 193, 'dcstim_start': 0, 'dcstim_total': 1000, 'dcstim_amp': 1129389296, 'device': None, 'xclbin': None, 'uuid': None, 'kernel_name': None, 'kernel': None}
Initialized kernel NeuroRing:

In [8]:
import time

start_time = time.time()
host.run_kernels(100000)
host.wait_for_kernels(100000)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

write done kernel <pyxrt.kernel object at 0x7f9bf1052c70>
Running kernel <pyxrt.kernel object at 0x7f9bf1052c70>
Running kernel <pyxrt.kernel object at 0x7f9bf58d1ab0>
write done kernel <pyxrt.kernel object at 0x7f9bf1052c30>
Running kernel <pyxrt.kernel object at 0x7f9bf1052c30>
Running kernel <pyxrt.kernel object at 0x7f9bf58fc5f0>
write done kernel <pyxrt.kernel object at 0x7f9bf1052cf0>
Running kernel <pyxrt.kernel object at 0x7f9bf1052cf0>
Running kernel <pyxrt.kernel object at 0x7f9bf1052d30>
write done kernel <pyxrt.kernel object at 0x7f9bf12dc370>
Running kernel <pyxrt.kernel object at 0x7f9bf12dc370>
Running kernel <pyxrt.kernel object at 0x7f9bf1052e70>
Axon loader run complete <pyxrt.kernel object at 0x7f9bf1052c70>
Kernel run complete <pyxrt.kernel object at 0x7f9bf58d1ab0>
Axon loader run complete <pyxrt.kernel object at 0x7f9bf1052c30>
Kernel run complete <pyxrt.kernel object at 0x7f9bf58fc5f0>
Axon loader run complete <pyxrt.kernel object at 0x7f9bf1052cf0>
Kernel run co

In [13]:
array = host.kernels_per_fpga[0][2].get_spike_recorder_array(10000)
print(array)
print(len(array))

for i in range(len(array)):
    if array[i] != 0:
        print(i)

[0 0 0 ... 0 0 0]
640000
2592
2720
2848
2976
3104
3232
3280
3360
3408
3488
3536
3616
3664
3744
3792
3872
3920
4000
4048
4128
4176
4256
4272
4304
4384
4400
4432
4480
4512
4528
4560
4608
4640
4656
4688
4736
4768
4784
4816
4864
4896
4912
4944
4992
5008
5024
5040
5072
5120
5136
5152
5168
5200
5248
5264
5280
5296
5312
5328
5376
5392
5408
5424
5440
5456
5504
5520
5536
5552
5568
5584
5632
5648
5664
5680
5696
5712
5760
5776
5792
5808
5824
5840
5888
5904
5920
5936
5952
5968
6016
6032
6048
6064
6080
6096
6144
6160
6176
6192
6208
6224
6272
6288
6304
6320
6336
6352
6400
6416
6432
6448
6464
6480
6528
6544
6560
6576
6592
6608
6656
6672
6688
6704
6720
6736
6784
6800
6816
6832
6848
6864
6912
6928
6944
6960
6976
6992
7040
7056
7072
7088
7104
7120
7168
7184
7200
7216
7232
7248
7296
7312
7328
7344
7360
7376
7424
7440
7456
7472
7488
7504
7552
7568
7584
7600
7616
7632
7680
7696
7712
7728
7744
7760
7808
7824
7840
7856
7872
7888
7936
7952
7968
7984
8000
8016
8064
8080
8096
8112
8128
8144
8192
8208
8224
8240


In [ ]:
print(len(host.packed_list_per_fpga[0][0]))
print(len(host.packed_list_per_fpga[0][1]))


In [ ]:
host.kernels_per_fpga[0][0].run_kernel(host.packed_list_per_fpga[0][0], 2)
host.kernels_per_fpga[0][1].run_kernel(host.packed_list_per_fpga[0][1], 2)

In [ ]:
host.kernels_per_fpga[0][0].wait_for_axon()

In [ ]:
host.kernels_per_fpga[0][1].wait_for_axon()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_neuroring()

In [ ]:
host.kernels_per_fpga[0][1].wait_for_neuroring()

In [ ]:
host.kernels_per_fpga[0][0].wait_for_kernel()
print(host.kernels[0].spikeRecorder_array)


In [ ]:
for i in range(len(host.kernels[0].spikeRecorder_array)):
    if host.kernels[0].spikeRecorder_array[i] > 0:
        print(i)

In [ ]:
print(host.kernels[0].spikeRecorder_array)


In [ ]:
print(host.kernel_neuron_ranges)

In [ ]:
print(net.DC_amp)
print(np.average(net.DC_amp))

In [ ]:
print(host.kernels_per_fpga[0][1].kernel.group_id(0))
print(host.kernels_per_fpga[0][1].kernel.group_id(1))
print(host.kernels_per_fpga[0][1].kernel.group_id(2))
print(host.kernels_per_fpga[0][1].kernel.group_id(3))
print(host.kernels_per_fpga[0][1].kernel.group_id(4))
print(host.kernels_per_fpga[0][1].kernel.group_id(5))
print(host.kernels_per_fpga[0][1].kernel.group_id(6))
print(host.kernels_per_fpga[0][1].kernel.group_id(7))
print(host.kernels_per_fpga[0][1].kernel.group_id(8))
print(host.kernels_per_fpga[0][1].kernel.group_id(9))
print(host.kernels_per_fpga[0][1].kernel.group_id(10))
print(host.kernels_per_fpga[0][1].kernel.group_id(11))
print(host.kernels_per_fpga[0][1].kernel.group_id(12))
print(host.kernels_per_fpga[0][1].kernel.group_id(13))
print(host.kernels_per_fpga[0][1].kernel.group_id(14))
print(host.kernels_per_fpga[0][1].kernel.group_id(15))
print(host.kernels_per_fpga[0][1].kernel.group_id(16))
print(host.kernels_per_fpga[0][1].kernel.group_id(17))
